In [2]:
#pdf loader
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("")
docs = loader.load()
docs

[Document(metadata={'source': 'cultivation.pdf', 'page': 0}, page_content='See discussions, st ats, and author pr ofiles f or this public ation at : https://www .researchgate.ne t/public ation/265385470\nBody Cultivation in Contemporary China\nChapt er · Januar y 2006\nCITATIONS\n4READS\n12,313\n1 author:\nSome o f the author s of this public ation ar e also w orking on these r elat ed pr ojects:\nDream T ripper s: Glob al Daoism and the Pr edic ament of Modern Spirt uality  View pr oject\nSelf -Government in China: Mor al Autonomy under Neo -Socialism  View pr oject\nDavid Ale xander P almer\nThe Univ ersity of Hong K ong\n123 PUBLICA TIONS \xa0\xa0\xa0686 CITATIONS \xa0\xa0\xa0\nSEE PROFILE\nAll c ontent f ollo wing this p age was uplo aded b y David Ale xander P almer  on 08 Sept ember 2014.\nThe user has r equest ed enhanc ement of the do wnlo aded file.'),
 Document(metadata={'source': 'cultivation.pdf', 'page': 1}, page_content='Body Cultivation in Contemporary China  David A. Pa

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_Splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=20)
documents = text_Splitter.split_documents(docs)
len(documents)

78

In [5]:
#vectorization and db
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

db = FAISS.from_documents(documents,OllamaEmbeddings())

In [6]:
#query
query = "What is cultivation?"
result = db.similarity_search(query)
result

[Document(metadata={'source': 'cultivation.pdf', 'page': 5}, page_content='body and mental discipline. Provincial Party leaders supported his work and called on him to found a qigong clinic in the city of Tangshan in 1953, followed two years later by the foundation of a qigong sanatorium in the exclusive seaside resort of Beidaihe, a favourite retreat for the Party elite. Liu Guizhen was honoured by Vice-President Liu Shaoqi and treated several of the country’s top leaders. Qigong was recognized as a discipline of Chinese medicine, alongside herbalism,                                                 6 See Kohn 2002.  7 On the etymology of the term qigong, see Despeux 1997: 267.'),
 Document(metadata={'source': 'cultivation.pdf', 'page': 3}, page_content='means of nurturing awareness of ultimate reality and, in the case of tantrism, of nurturing and controlling divine powers. The Shaolin and Wudang traditions of martial arts developed the use of body cultivation as the foundation of tec

In [7]:
#llm
from langchain_community.llms import Ollama
llm = Ollama(model="llama2")
llm

Ollama()

In [8]:
#prompt
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(""""
Answer the following question based only on the provided context.
Answer as detailed as possible.
<context>
{context}
</context>
Question:{input} """
)

In [9]:
#chain
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm,prompt)


In [10]:
#retrievers
retriever = db.as_retriever()

#combining chain -- Retriver chain
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(retriever,document_chain)

In [11]:
#querying

response = retrieval_chain.invoke({"input":"Give me a brief , what this pdf is about"})
response

{'input': 'Give me a brief , what this pdf is about',
 'context': [Document(metadata={'source': 'cultivation.pdf', 'page': 5}, page_content='Such a process began during the Republican era (1911 – 1949), when some authors popularised sitting meditation and other practices through widely circulated books that sought to eliminate the obscure esoteric language in which the techniques had traditionally been couched, and to present them in the idioms of psychology, physiology, and physics6. But it was under the Communist regime that body cultivation became the subject of a distinct category as part of a larger project of state expansion.   The birth of qigong   During the Chinese civil and anti-Japanese wars (1927-1949), Red Army and Communist Party units, retrenched in remote “Liberated Areas” and lacking access to modern medical facilities, encouraged the use of traditional local remedies for the care of injured and ill soldiers and officials. In was under such conditions that cadre Liu Gu

In [12]:
response['answer']

"Sure! The PDF appears to be an academic essay discussing the history and development of qigong, a form of traditional Chinese medicine and exercise. The essay covers the origins of qigong in ancient China, its popularization during the Republican era (1911-1949), and its recognition as a discipline of Chinese medicine alongside herbalism in 1989. The essay also touches on controversies surrounding qigong, including accusations of psychological suggestion and self-hypnosis, as well as the government's response to these criticisms through licensing qigong masters and taxing qigong organizations. Overall, the PDF provides a detailed overview of the history and development of qigong in China."